## Tutorial: Computer Vision Model Training

### University of California, Santa Barbara  
### PSTAT 135/235  
### Last Updated: Oct 22, 2018

---  

### OBJECTIVES
- Use keras to apply transfer learning to a convolutional neural net model for a computer vision task

---  


## Import Modules

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Dense
from sklearn import metrics
import time
import numpy as np
import os
import sys
import pickle
import pandas as pd
import json
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [5]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [1]:
import requests 

## Function Definitions

In [12]:
def setup_model_for_transfer_learning(src_model, size, pooling, dropout_fraction, learn_rate):
    '''
    model setup for transfer learning based on input source model
    
    INPUT
    src_model         keras app
    size              tuple(int) 
    pooling           str
    dropout_fraction  float, dropout fraction
    learn_rate        float, learning rate
    
    OUTPUT
    model
    '''

    # fixed parameters
    neurons_in_dense_layer_1 = 1024
    loss_fcn = 'categorical_crossentropy'
    output_classes = 2
    
    target_size = size + (3,)
    source_model = src_model(input_shape=target_size, pooling=pooling, include_top=False, weights='imagenet')
    
    for layer in src_model.layers:
        layer.trainable=False # this freezes layers for transfer learning
    
    # keras allows for sequentially adding layers to the source model
    model = Sequential()
    model.add(src_model)
    model.add(Dense(neurons_in_dense_layer, activation='relu')) # rectified linear unit
    model.add(Dropout(dropout_fraction))
    model.add(Dense(output_classes, activation='softmax'))
    opt = optimizers.Adam(lr = learn_rate)
    model.compile(loss=loss_fcn, optimizer=opt, metrics=['accuracy'])

    return model

## Parameters

In [4]:
train_data_dir = # enter path to training data here
epochs = 50
dropout_frac = 0.50
batch_size = 100
lr = 0.05
pooling = 'max' # check this is vaid
img_height = 224
img_width = 244
keras_app = applications.VGG16

In [8]:
# determine train_samples here
train_steps = train_samples / batch_size

## Model setup

In [5]:
print('\n==== setting up model...')
model = setup_model_for_transfer_learning(source_model=keras_app, 
                                        size=(img_height, img_width),
                                        pooling = pooling,
                                        dropout_fraction=dropout_frac,
                                        learn_rate=lr)
print('\n==== completed setting up model.')

## Data setup

In [6]:
train_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

## Fit Model

In [ ]:
print('\n==== begin fitting model...')
model_train_start_time = time.time()
model.fit_generator(train_generator,
                    steps_per_epoch=train_steps,
                    epochs=epochs)